In [143]:
import pandas as pd
import numpy as np

train_data = pd.read_csv("data/train.csv")
train_data = train_data.drop(['Name', 'Cabin', 'Ticket'], axis=1)
test_data = pd.read_csv("data/test.csv")
test_data = test_data.drop(['Name', 'Cabin', 'Ticket'], axis=1)

con = train_data['Sex'] == 'female'
con2 = train_data['Sex'] == 'male'

train_data.loc[con, 'Age'] = train_data.loc[con, 'Age'].fillna(train_data.loc[con, 'Age'].mean())
train_data.loc[con2, 'Age'] = train_data.loc[con2, 'Age'].fillna(train_data.loc[con2, 'Age'].mean())
train_data['Embarked'] = train_data['Embarked'].fillna('S')

test_data.loc[con, 'Age'] = test_data.loc[con, 'Age'].fillna(test_data.loc[con, 'Age'].mean())
test_data.loc[con2, 'Age'] = test_data.loc[con2, 'Age'].fillna(test_data.loc[con2, 'Age'].mean())
test_data['Embarked'] = test_data['Embarked'].fillna('S')

l20 = []
l40 = []
l60 = []
l80 = []

for l in train_data['Age']:
    if 0 <= l <= 20:
        l20.append(True)
        l40.append(False)
        l60.append(False)
        l80.append(False)
    elif 20 < l <= 40:
        l20.append(False)
        l40.append(True)
        l60.append(False)
        l80.append(False)
    elif 40 < l <= 60:
        l20.append(False)
        l40.append(False)
        l60.append(True)
        l80.append(False)
    else:
        l20.append(False)
        l40.append(False)
        l60.append(False)
        l80.append(True)
tl20 = []
tl40 = []
tl60 = []
tl80 = []
for l in test_data['Age']:
    if 0 <= l <= 20:
        tl20.append(True)
        tl40.append(False)
        tl60.append(False)
        tl80.append(False)
    elif 20 < l <= 40:
        tl20.append(False)
        tl40.append(True)
        tl60.append(False)
        tl80.append(False)
    elif 40 < l <= 60:
        tl20.append(False)
        tl40.append(False)
        tl60.append(True)
        tl80.append(False)
    else:
        tl20.append(False)
        tl40.append(False)
        tl60.append(False)
        tl80.append(True)

train_data['Age_0_20'] = l20
train_data['Age_20_40'] = l40
train_data['Age_40_60'] = l60
train_data['Age_60+'] = l80

test_data['Age_0_20'] = tl20
test_data['Age_20_40'] = tl40
test_data['Age_40_60'] = tl60
test_data['Age_60+'] = tl80


In [145]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

train_target = train_data["Survived"]
features = ['Age_0_20','Age_20_40','Age_40_60','Age_60+' , 'Fare', 'Pclass', 'Sex']

train_input = pd.get_dummies(train_data[features])
test_input = pd.get_dummies(test_data[features])

print(train_input)
model = RandomForestClassifier(oob_score = True, max_depth= 10, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 5, n_estimators= 100, random_state= 42)
model.fit(train_input, train_target)
predictions = model.predict(test_input)
scores = cross_validate(model, train_input, train_target, return_train_score = True, n_jobs = 1)
print(scores)
print(model.oob_score_)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

     Age_0_20  Age_20_40  Age_40_60  Age_60+     Fare  Pclass  Sex_female  \
0       False       True      False    False   7.2500       3       False   
1       False       True      False    False  71.2833       1        True   
2       False       True      False    False   7.9250       3        True   
3       False       True      False    False  53.1000       1        True   
4       False       True      False    False   8.0500       3       False   
..        ...        ...        ...      ...      ...     ...         ...   
886     False       True      False    False  13.0000       2       False   
887      True      False      False    False  30.0000       1        True   
888     False       True      False    False  23.4500       3        True   
889     False       True      False    False  30.0000       1       False   
890     False       True      False    False   7.7500       3       False   

     Sex_male  
0        True  
1       False  
2       False  
3       Fal